In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
from aif360.algorithms.preprocessing import Reweighing

/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset, predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attributes):
    features = [privileged_attribute] + unprivileged_attributes

    aif_race_dataset = BinaryLabelDataset(
            df=dataset,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    aif_race_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    race_privileged_group = [{privileged_attribute: 1}]
    race_unprivileged_groups = [{attr: 1} for attr in unprivileged_attributes]

    fairness_metrics = ClassificationMetric(dataset=aif_race_dataset,
                            classified_dataset=aif_race_pred,
                            unprivileged_groups=race_unprivileged_groups,
                            privileged_groups=race_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Average Odds Difference (AOD): {AOD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")

In [3]:
dataset_path = 'tidyAdult-trained.csv'
df_raw = pd.read_csv(dataset_path)
pd.set_option('display.max_columns', None)
df_raw.head()

,age,fnlwgt,education-num,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,sex_Female
0,50,288,13,13,0,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,38,464,9,40,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,53,484,7,40,0,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,28,581,13,40,0,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,Fal

In [5]:
X = df_raw.drop(columns="income")
y = df_raw["income"]

privileged_attributeSex = "sex_Male"
unprivileged_attributesSex = ["sex_Female"]
sex_features = [privileged_attributeSex] + unprivileged_attributesSex

privileged_attributeRace = "race_White"
unprivileged_attributesRace = ["race_Black", "race_Other", "race_Asian-Pac-Islander"]
race_features = [privileged_attributeRace] + unprivileged_attributesRace

In [6]:
privileged_group = [{privileged_attributeSex: 1}]
unprivileged_groups = [{attr: 1} for attr in unprivileged_attributesSex]

dataset = X.copy(deep=True) # we recreate a copy of the whole dataset processed and balanced
dataset['income'] = y # and join the target feature with the others

# Create a BinaryLabelDataset using AIF360 library
aif_dataset = BinaryLabelDataset(
        df=dataset,
        favorable_label=1,
        unfavorable_label=0,
        label_names=['income'],
        protected_attribute_names=sex_features,
    )

# Apply the Reweighing technique
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_group)
dataset_transformed = RW.fit_transform(aif_dataset)

# Get sample weights from the transformed dataset
sample_weights = dataset_transformed.instance_weights

# Create a fair dataset using the reweighing technique
fair_dataset = dataset.copy(deep=True)
fair_dataset['weights'] = sample_weights

# Get features and target from the dataset
features = dataset.columns.tolist()
features.remove('income')
target = ['income']

# Set dataset features and target
X_fair = fair_dataset[features]
y_fair = fair_dataset[target]

X_fair_train, X_fair_test, y_fair_train, y_fair_test, sample_weights_train, sample_weights_test = train_test_split(X_fair, y_fair, sample_weights, test_size=0.2, random_state=42)

In [7]:
def train_models(sample_weight_tr):
    dt_clf = DecisionTreeClassifier(random_state=42)

    # The fit function will do the trick
    dt_clf.fit(X_fair_train, y_fair_train, sample_weight=sample_weight_tr)

    # After the training phase, the model will be tested by predicting the values on the test set
    dt_predictions = dt_clf.predict(X_fair_test)

    svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

    # Addestro il classificatore sui dati di training
    svm_classifier.fit(X_fair_train, y_fair_train, svc__sample_weight=sample_weight_tr)

    # predizioni sui dati di test
    svm_pred = svm_classifier.predict(X_fair_test)

    rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

    rf_classifier.fit(X_fair_train, y_fair_train, sample_weight=sample_weight_tr)

    rf_predictions = rf_classifier.predict(X_fair_test)

    return dt_predictions,svm_pred,rf_predictions

In [8]:
dt_predictions, svm_pred, rf_predictions = train_models(sample_weights_train)

/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [9]:
compute_performance_metrics(y_fair_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.7945118866970157
Decision Tree Recall: 0.49746192893401014
Decision Tree F1 Score: 0.49107422486689634


In [10]:
compute_performance_metrics(y_fair_test,svm_pred,"SVM")

SVM Accuracy: 0.8225847243297926
SVM Recall: 0.27220812182741116
SVM F1 Score: 0.37947810703228657


In [11]:
compute_performance_metrics(y_fair_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.840288315629742
Random Forest Recall: 0.48984771573604063
Random Forest F1 Score: 0.5500534378339864


In [12]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_fair_test.copy(deep=True) # we create a copy of the test set
dataset['income'] = y_fair_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['income'] = dt_predictions # but this time the target feature is made by the predictions of our model

In [13]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.17
Average Odds Difference (AOD): -0.111
Equal Opportunity Difference (EOD): -0.11


In [14]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L'SVM
dataset = X_fair_test.copy(deep=True)
dataset['income'] = y_fair_test
predictions = dataset.copy(deep=True) 
predictions['income'] = svm_pred

In [15]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.06
Average Odds Difference (AOD): 0.268
Equal Opportunity Difference (EOD): 0.484


In [16]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_fair_test.copy(deep=True)
dataset['income'] = y_fair_test
predictions = dataset.copy(deep=True) 
predictions['income'] = rf_predictions

In [17]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.156
Average Odds Difference (AOD): -0.092
Equal Opportunity Difference (EOD): -0.096


LAVORO CON RACE

In [18]:
privileged_group = [{privileged_attributeRace: 1}]
unprivileged_groups = [{attr: 1} for attr in unprivileged_attributesRace]

dataset = X.copy(deep=True) # we recreate a copy of the whole dataset processed and balanced
dataset['income'] = y # and join the target feature with the others

# Create a BinaryLabelDataset using AIF360 library
aif_dataset = BinaryLabelDataset(
        df=dataset,
        favorable_label=1,
        unfavorable_label=0,
        label_names=['income'],
        protected_attribute_names=race_features,
    )

# Apply the Reweighing technique
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_group)
dataset_transformed = RW.fit_transform(aif_dataset)

# Get sample weights from the transformed dataset
sample_weights = dataset_transformed.instance_weights

# Create a fair dataset using the reweighing technique
fair_dataset = dataset.copy(deep=True)
fair_dataset['weights'] = sample_weights

# Get features and target from the dataset
features = dataset.columns.tolist()
features.remove('income')
target = ['income']

# Set dataset features and target
X_fair = fair_dataset[features]
y_fair = fair_dataset[target]

X_fair_train, X_fair_test, y_fair_train, y_fair_test, sample_weights_train, sample_weights_test = train_test_split(X_fair, y_fair, sample_weights, test_size=0.2, random_state=42)

In [19]:
dt_predictions, svm_pred, rf_predictions = train_models(sample_weights_train)

/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [20]:
compute_performance_metrics(y_fair_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.7947647951441578
Decision Tree Recall: 0.49238578680203043
Decision Tree F1 Score: 0.4888188976377953


In [21]:
compute_performance_metrics(y_fair_test,svm_pred,"SVM")

SVM Accuracy: 0.8376327769347496
SVM Recall: 0.39911167512690354
SVM F1 Score: 0.4948859166011015


In [22]:
compute_performance_metrics(y_fair_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.8405412240768841
Random Forest Recall: 0.48984771573604063
Random Forest F1 Score: 0.550445632798574


In [23]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_fair_test.copy(deep=True) # we create a copy of the test set
dataset['income'] = y_fair_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['income'] = dt_predictions # but this time the target feature is made by the predictions of our model

In [24]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.079
Average Odds Difference (AOD): -0.049
Equal Opportunity Difference (EOD): -0.053


In [25]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L'SVM
dataset = X_fair_test.copy(deep=True)
dataset['income'] = y_fair_test
predictions = dataset.copy(deep=True) 
predictions['income'] = svm_pred

In [26]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.043
Average Odds Difference (AOD): 0.008
Equal Opportunity Difference (EOD): 0.033


In [27]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_fair_test.copy(deep=True)
dataset['income'] = y_fair_test
predictions = dataset.copy(deep=True) 
predictions['income'] = rf_predictions

In [28]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.08
Average Odds Difference (AOD): -0.045
Equal Opportunity Difference (EOD): -0.049
